# NRMS: Neural News Recommendation with Multi-Head Self-Attention
NRMS \[1\] is a neural news recommendation approach with multi-head selfattention. The core of NRMS is a news encoder and a user encoder. In the newsencoder, a multi-head self-attentions is used to learn news representations from news titles by modeling the interactions between words. In the user encoder, we learn representations of users from their browsed news and use multihead self-attention to capture the relatedness between the news. Besides, we apply additive
attention to learn more informative news and user representations by selecting important words and news.

## Properties of NRMS:
- NRMS is a content-based neural news recommendation approach.
- It uses multi-self attention to learn news representations by modeling the iteractions between words and learn user representations by capturing the relationship between user browsed news.
- NRMS uses additive attentions to learn informative news and user representations by selecting important words and news.

## Data format:
For quicker training and evaluaiton, we sample MINDdemo dataset of 5k users from [MIND small dataset](https://msnews.github.io/). The MINDdemo dataset has the same file format as MINDsmall and MINDlarge. If you want to try experiments on MINDsmall and MINDlarge, please change the dowload source. Select the MIND_type parameter from ['large', 'small', 'demo'] to choose dataset.
 
**MINDdemo_train** is used for training, and **MINDdemo_dev** is used for evaluation. Training data and evaluation data are composed of a news file and a behaviors file. You can find more detailed data description in [MIND repo](https://github.com/msnews/msnews.github.io/blob/master/assets/doc/introduction.md)

### news data
This file contains news information including newsid, category, subcatgory, news title, news abstarct, news url and entities in news title, entities in news abstarct.
One simple example: <br>

`N46466	lifestyle	lifestyleroyals	The Brands Queen Elizabeth, Prince Charles, and Prince Philip Swear By	Shop the notebooks, jackets, and more that the royals can't live without.	https://www.msn.com/en-us/lifestyle/lifestyleroyals/the-brands-queen-elizabeth,-prince-charles,-and-prince-philip-swear-by/ss-AAGH0ET?ocid=chopendata	[{"Label": "Prince Philip, Duke of Edinburgh", "Type": "P", "WikidataId": "Q80976", "Confidence": 1.0, "OccurrenceOffsets": [48], "SurfaceForms": ["Prince Philip"]}, {"Label": "Charles, Prince of Wales", "Type": "P", "WikidataId": "Q43274", "Confidence": 1.0, "OccurrenceOffsets": [28], "SurfaceForms": ["Prince Charles"]}, {"Label": "Elizabeth II", "Type": "P", "WikidataId": "Q9682", "Confidence": 0.97, "OccurrenceOffsets": [11], "SurfaceForms": ["Queen Elizabeth"]}]	[]`
<br>

In general, each line in data file represents information of one piece of news: <br>

`[News ID] [Category] [Subcategory] [News Title] [News Abstrct] [News Url] [Entities in News Title] [Entities in News Abstract] ...`

<br>

We generate a word_dict file to tranform words in news title to word indexes, and a embedding matrix is initted from pretrained glove embeddings.

### behaviors data
One simple example: <br>
`1	U82271	11/11/2019 3:28:58 PM	N3130 N11621 N12917 N4574 N12140 N9748	N13390-0 N7180-0 N20785-0 N6937-0 N15776-0 N25810-0 N20820-0 N6885-0 N27294-0 N18835-0 N16945-0 N7410-0 N23967-0 N22679-0 N20532-0 N26651-0 N22078-0 N4098-0 N16473-0 N13841-0 N15660-0 N25787-0 N2315-0 N1615-0 N9087-0 N23880-0 N3600-0 N24479-0 N22882-0 N26308-0 N13594-0 N2220-0 N28356-0 N17083-0 N21415-0 N18671-0 N9440-0 N17759-0 N10861-0 N21830-0 N8064-0 N5675-0 N15037-0 N26154-0 N15368-1 N481-0 N3256-0 N20663-0 N23940-0 N7654-0 N10729-0 N7090-0 N23596-0 N15901-0 N16348-0 N13645-0 N8124-0 N20094-0 N27774-0 N23011-0 N14832-0 N15971-0 N27729-0 N2167-0 N11186-0 N18390-0 N21328-0 N10992-0 N20122-0 N1958-0 N2004-0 N26156-0 N17632-0 N26146-0 N17322-0 N18403-0 N17397-0 N18215-0 N14475-0 N9781-0 N17958-0 N3370-0 N1127-0 N15525-0 N12657-0 N10537-0 N18224-0`
<br>

In general, each line in data file represents one instance of an impression. The format is like: <br>

`[Impression ID] [User ID] [Impression Time] [User Click History] [Impression News]`

<br>

User Click History is the user historical clicked news before Impression Time. Impression News is the displayed news in an impression, which format is:<br>

`[News ID 1]-[label1] ... [News ID n]-[labeln]`

<br>
Label represents whether the news is clicked by the user. All information of news in User Click History and Impression News can be found in news data file.

## Global settings and imports

In [3]:
import sys
path = '/home/rain/mind/NRMS'
sys.path.append(path + '/recsys')
import os
import numpy as np
import zipfile
from tqdm import tqdm
#import scrapbook as sb
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

#from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources 
from recommenders.models.newsrec.newsrec_utils import prepare_hparams
from recommenders.models.newsrec.models.nrms import NRMSModel
from recommenders.models.newsrec.io.mind_iterator import MINDIterator
from recommenders.models.newsrec.newsrec_utils import get_mind_data_set

print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))
import recommenders.models.newsrec.io.mind_iterator as mind_tt
import importlib
importlib.reload(mind_tt)

ModuleNotFoundError: No module named 'recommenders.datasets'

## Prepare parameters

In [2]:
epochs = 1
seed = 42
batch_size = 32


## Download and load data

In [3]:
#tmpdir = TemporaryDirectory()
data_path = "/home/rain/data" #tmpdir.name

train_dir = os.path.join(data_path, 'MINDsmall_train')
train_news_file = os.path.join(train_dir, r'news.tsv')
train_behaviors_file = os.path.join(train_dir, r'behaviors.tsv')

valid_dir = os.path.join(data_path, 'MINDsmall_dev')
valid_news_file = os.path.join(valid_dir, r'news.tsv')
valid_behaviors_file = os.path.join(valid_dir, r'behaviors.tsv')


wordEmb_file = os.path.join(data_path, "utils", "embedding.npy")
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path, "utils", "word_dict.pkl")
yaml_file = os.path.join(data_path, "utils", r'nrms.yaml')


## Create hyper-parameters

In [4]:
hparams = prepare_hparams(yaml_file, 
                          wordEmb_file=wordEmb_file,
                          wordDict_file=wordDict_file, 
                          userDict_file=userDict_file,
                          batch_size=batch_size,
                          epochs=epochs,
                          show_step=10)
print(hparams)

data_format=news,iterator_type=None,support_quick_scoring=True,wordEmb_file=/data/ebay/notebooks/knie/mind/utils/embedding.npy,wordDict_file=/data/ebay/notebooks/knie/mind/utils/word_dict.pkl,userDict_file=/data/ebay/notebooks/knie/mind/utils/uid2index.pkl,vertDict_file=None,subvertDict_file=None,title_size=30,body_size=None,word_emb_dim=300,word_size=None,user_num=None,vert_num=None,subvert_num=None,his_size=50,npratio=4,dropout=0.2,attention_hidden_dim=200,head_num=20,head_dim=20,cnn_activation=None,dense_activation=None,filter_num=200,window_size=3,vert_emb_dim=100,subvert_emb_dim=100,gru_unit=400,type=ini,user_emb_dim=50,learning_rate=0.0001,loss=cross_entropy_loss,optimizer=adam,epochs=1,batch_size=32,show_step=10,metrics=['group_auc', 'mean_mrr', 'ndcg@5;10']


## Train the NRMS model

In [5]:
iterator = MINDIterator

In [6]:
model = NRMSModel(hparams,  iterator  , seed=seed)

2021-11-25 05:03:31.237644: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2021-11-25 05:03:31.253550: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2100000000 Hz
2021-11-25 05:03:31.257321: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55d8568999d0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-11-25 05:03:31.257361: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-11-25 05:03:31.259371: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-11-25 05:03:31.528350: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55d85689be40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2021-11-25 05:03:31.528388: I tensorflow/compil

In [7]:
print(model.run_eval(valid_news_file, valid_behaviors_file))

0it [00:00, ?it/s]2021-11-25 05:03:43.576735: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2251it [00:07, 308.63it/s]
11765it [02:34, 76.17it/s]
376471it [00:56, 6649.87it/s] 


{'group_auc': 0.4834, 'mean_mrr': 0.2083, 'ndcg@5': 0.2085, 'ndcg@10': 0.2744}


In [9]:
group_impr_indexes, group_labels, group_preds = model.run_fast_eval(valid_news_file, valid_behaviors_file)

2251it [00:01, 1205.91it/s]
11765it [02:13, 88.30it/s]
376471it [00:53, 7004.29it/s] 


In [ ]:
%%time
model.fit(train_news_file, train_behaviors_file, valid_news_file, valid_behaviors_file)

step 105740 , total_loss: 1.2578, data_loss: 1.5218: : 105740it [1:21:20, 21.67it/s]
2251it [00:04, 472.41it/s]
11765it [02:45, 70.94it/s]
376471it [01:02, 5979.66it/s]


at epoch 1
train info: logloss loss:1.257834349770232
eval info: group_auc:0.6767, mean_mrr:0.3264, ndcg@10:0.4246, ndcg@5:0.3589
at epoch 1 , train time: 4880.5 eval time: 528.8


step 105740 , total_loss: 1.2010, data_loss: 1.1799: : 105740it [1:19:12, 22.25it/s]
2251it [00:02, 1037.58it/s]
11765it [02:14, 87.35it/s]
376471it [01:02, 5983.74it/s]


at epoch 2
train info: logloss loss:1.2009898208846932
eval info: group_auc:0.6729, mean_mrr:0.3205, ndcg@10:0.4202, ndcg@5:0.3536
at epoch 2 , train time: 4752.7 eval time: 495.8


step 105740 , total_loss: 1.1833, data_loss: 0.8421: : 105740it [1:19:17, 22.23it/s]
2251it [00:02, 1043.80it/s]
11765it [02:14, 87.23it/s]
376471it [01:02, 5991.63it/s]


at epoch 3
train info: logloss loss:1.1832750963605554
eval info: group_auc:0.6838, mean_mrr:0.3307, ndcg@10:0.4306, ndcg@5:0.3652
at epoch 3 , train time: 4757.2 eval time: 496.7


step 105740 , total_loss: 1.1720, data_loss: 1.2137: : 105740it [1:19:14, 22.24it/s]
2251it [00:02, 1039.98it/s]
11765it [02:14, 87.24it/s]
376471it [01:03, 5897.89it/s]


at epoch 4
train info: logloss loss:1.1720186772425638
eval info: group_auc:0.6816, mean_mrr:0.3309, ndcg@10:0.4309, ndcg@5:0.3649
at epoch 4 , train time: 4754.7 eval time: 503.8


step 96940 , total_loss: 1.1629, data_loss: 1.1446: : 96939it [1:12:41, 22.37it/s]

In [28]:
%%time
res_syn = model.run_eval(valid_news_file, valid_behaviors_file)
print(res_syn)


2251it [00:02, 931.58it/s] 
11765it [02:18, 85.18it/s]
376471it [01:05, 5772.60it/s]


{'group_auc': 0.6818, 'mean_mrr': 0.3288, 'ndcg@5': 0.3628, 'ndcg@10': 0.4291}
CPU times: user 14min 37s, sys: 1h 1min 23s, total: 1h 16min
Wall time: 8min 21s


In [14]:
#sb.glue("res_syn", res_syn)
print(res_syn)

hello world


## Save the model

In [29]:
model_path = os.path.join(data_path, "model")
os.makedirs(model_path, exist_ok=True)

model.model.save_weights(os.path.join(model_path, "nrms_ckpt"))

## Output Predcition File
This code segment is used to generate the prediction.zip file, which is in the same format in [MIND Competition Submission Tutorial](https://competitions.codalab.org/competitions/24122#learn_the_details-submission-guidelines).

Please change the `MIND_type` parameter to `large` if you want to submit your prediction to [MIND Competition](https://msnews.github.io/competition.html).

In [46]:
model.test_iterator = mind_tt.MINDIterator(
            hparams,
            col_spliter="\t",
        )

In [47]:
test_news_file = os.path.join(data_path, 'test', r'news.tsv')
test_behaviors_file = os.path.join(data_path, 'test', r'behaviors.tsv')

In [41]:
#a= model.run_fast_eval(valid_news_file)

2251it [00:02, 961.41it/s]


In [42]:
#len(a)

72024

In [48]:
group_impr_indexes, group_labels, group_preds = model.run_fast_eval(test_news_file, test_behaviors_file)

3781it [00:06, 612.28it/s]
74086it [16:19, 75.60it/s]
2370727it [07:52, 5021.96it/s]


In [49]:
mode

2370727

In [50]:
with open(os.path.join(data_path, 'prediction.txt'), 'w') as f:
    for impr_index, preds in tqdm(zip(group_impr_indexes, group_preds)):
        impr_index += 1
        pred_rank = (np.argsort(np.argsort(preds)[::-1]) + 1).tolist()
        pred_rank = '[' + ','.join([str(i) for i in pred_rank]) + ']'
        f.write(' '.join([str(impr_index), pred_rank])+ '\n')

2370727it [00:39, 59638.31it/s]


In [51]:
f = zipfile.ZipFile(os.path.join(data_path, 'prediction.zip'), 'w', zipfile.ZIP_DEFLATED)
f.write(os.path.join(data_path, 'prediction.txt'), arcname='prediction.txt')
f.close()

## Reference
\[1\] Wu et al. "Neural News Recommendation with Multi-Head Self-Attention." in Proceedings of the 2019 Conference on Empirical Methods in Natural Language Processing and the 9th International Joint Conference on Natural Language Processing (EMNLP-IJCNLP)<br>
\[2\] Wu, Fangzhao, et al. "MIND: A Large-scale Dataset for News Recommendation" Proceedings of the 58th Annual Meeting of the Association for Computational Linguistics. https://msnews.github.io/competition.html <br>
\[3\] GloVe: Global Vectors for Word Representation. https://nlp.stanford.edu/projects/glove/